In [1]:
# Import dependencies
import os
import subprocess
import json
from pprint import pprint
from dotenv import load_dotenv

# Load and set environment variables
load_dotenv()
mnemonic=os.getenv("mnemonic")

# Import constants.py and necessary functions from bit and web3
from constants import *

from bit.network import NetworkAPI
from bit import PrivateKeyTestnet
from bit import wif_to_key

from web3 import Web3, middleware, Account
from web3.middleware import geth_poa_middleware

from web3.gas_strategies.time_based import medium_gas_price_strategy

#w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))
w3 = Web3(Web3.HTTPProvider('http://localhost:8545'))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)
w3.eth.setGasPriceStrategy(medium_gas_price_strategy)

from coincurve.keys import PrivateKey

In [2]:
# Create a function called `derive_wallets`
def derive_wallets(coin=BTC, mnemonic=mnemonic, depth=3):
    command = f'php ./derive -g --mnemonic="{mnemonic}" --cols=all --coin={coin} --numderive={depth} --format=json'
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()
    p_status = p.wait()
    # keys = json.loads(output)
    return json.loads(output)

# Create a dictionary object called coins to store the output from `derive_wallets`.

#coins = {'BTCTEST': derive_wallets(coin=BTC, mnemonic, depth=3), 'ETH': derive_wallets(coin=ETH, mnemonic, depth=3), 'BTCTEST': derive_wallets(coin=BTCTEST, mnemonic, depth=3)}
#print(output)

coins = {
    ETH: derive_wallets(coin=ETH),
    BTCTEST: derive_wallets(coin=BTCTEST),
}
coins

{'eth': [{'path': "m/44'/60'/0'/0/0",
   'address': '0xACD5650F73716e4644E565920677d711CBcaa8d5',
   'xprv': 'xprvA3YBquoaiQxUApZfK9DwA1ziNP8WnbCvXFyGKUiBE8H86iRyWsg9MDT6UA8J1CeAizjhWwqs8xSC3UsKBErr7UhAihFcYCdSpfp2JWfiJVb',
   'xpub': 'xpub6GXYFRLUYnWmPJe8RAkwX9wSvQy1C3vmtUts7s7nnTp6yWm84QzPu1maKQBvj7XsXRRWeFGxN7SUFaXnjkyDqFkv4W3en3rwWSfeZrFc1ju',
   'privkey': '0xd6fc22e07ab6249f9afa39a1143ee68f183311e2478df1350e48a5d82ebf87e9',
   'pubkey': '0257872142df3da8e966d2ad2d1fe3c222268e43c04e58f13bf05b4eece2aa7ba2',
   'pubkeyhash': '321a9150fe83442d89fd875ccb61a03fc33c2675',
   'index': 0},
  {'path': "m/44'/60'/0'/0/1",
   'address': '0x3cd4713cb58FCE3dC14f0527E19EA2A451F4c787',
   'xprv': 'xprvA3YBquoaiQxUDFdgaqGRPZCGHMe3cBQg87HhenkN4Sjx2Towu4ZJie3vbwRepXDoKn8H6hE89Vf3CLz5w2cZW86BYbPHN5Toh9ijQqgHhpe',
   'xpub': 'xpub6GXYFRLUYnWmRji9groRkh8zqPUY1e8XVLDJTB9ycnGvuG96SbsZGSNQTDP4AkQgtFNTtE5Y4AbXGMbPSVnbQmZDT9Yg4sdojcPrboMj8z6',
   'privkey': '0x338964724443f8e65f6812d9888fb21fb9d577a69e199d

In [3]:
# Create a function called `priv_key_to_account` that converts privkey strings to account objects.
def priv_key_to_account(coin, priv_key):
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    if coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)
    
acct = priv_key_to_account(ETH, '0xa50211e1c74190d6a344c1e646f8e3bf9bc2591815c196a8bcb0e11730cf12c4')
print(acct)

In [4]:
btcacct = priv_key_to_account(BTCTEST, 'cT7TPYw2pAVCCbpBKkX9cUxxNMq4iajwvNwHTkEzZLPx8wMHXMpn')
print(acct)

In [5]:
# Create a function called `create_tx` that creates an unsigned transaction appropriate metadata.
def create_tx(coin, account, to, amount):
    if coin == ETH:
        value = w3.toWei(amount, "ether") # convert 1.2 ETH to 120000000000 wei
        gasEstimate = w3.eth.estimateGas({ "to": to, "from": account, "amount": value })
        return {
            "to": to,
            "from": account,
            "value": value,
            "gas": gasEstimate,
            "gasPrice": w3.eth.generateGasPrice(),
            "nonce": w3.eth.getTransactionCount(account),
            "chainId": w3.eth.chain_id
        }
    if coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(to, amount, BTC)])


In [6]:
# Create a function called `send_tx` that calls `create_tx`, signs and sends the transaction.
def send_tx(coin, account, to, amount):
    if coin == ETH:
        raw_tx = create_tx(coin, account.address, to, amount)
        signed = account.signTransaction(raw_tx)
        return w3.eth.sendRawTransaction(signed.rawTransaction)
    if coin == BTCTEST:
        raw_tx = create_tx(coin, account, to, amount)
        signed = account.sign_transaction(raw_tx)
        return NetworkAPI.broadcast_tx_testnet(signed)
    

In [7]:
send_tx(BTCTEST, btcacct, 'mqfEdw57dF1kiVALzy1kjHSMpn2pKNFe6q', 0.0001)

In [11]:
send_tx(ETH, acct, '0xACD5650F73716e4644E565920677d711CBcaa8d5', 10000000000000000000)

HexBytes('0x1caebf861891ecf4b80586720f747ffc5146c220f68a29341e4c1ad20e6cdd64')